# Data Integrity:
Ensuring the accuracy, consistency, and reliability of data throughout the
cleaning process.

In [1]:
import pandas as pd

# Load the dataset
file_path = "/AB_NYC_2019.csv"
df = pd.read_csv(file_path)

# Show basic info and first few rows
basic_info = df.info()
head = df.head()

# Count missing values
missing_values = df.isnull().sum()

# Identify duplicates
duplicate_count = df.duplicated().sum()

df.shape, missing_values, duplicate_count, head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

((48895, 16),
 id                                    0
 name                                 16
 host_id                               0
 host_name                            21
 neighbourhood_group                   0
 neighbourhood                         0
 latitude                              0
 longitude                             0
 room_type                             0
 price                                 0
 minimum_nights                        0
 number_of_reviews                     0
 last_review                       10052
 reviews_per_month                 10052
 calculated_host_listings_count        0
 availability_365                      0
 dtype: int64,
 np.int64(0),
      id                                              name  host_id  \
 0  2539                Clean & quiet apt home by the park     2787   
 1  2595                             Skylit Midtown Castle     2845   
 2  3647               THE VILLAGE OF HARLEM....NEW YORK !     4632   
 3  3831         

# Missing Data Handling:
Dealing with missing values by either imputing them or making
informed decisions on how to handle gaps in the dataset.

In [2]:
# Fill missing 'name' and 'host_name' with "Unknown"
df['name'] = df['name'].fillna("Unknown")
df['host_name'] = df['host_name'].fillna("Unknown")

# Ensure 'last_review' is datetime
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

# Drop rows where 'last_review' is null
df = df.dropna(subset=['last_review'])

# Confirm all missing values are handled now
cleaned_missing = df.isnull().sum()

# Show updated data types and a few rows to confirm
dtypes = df.dtypes
sample_rows = df[['name', 'host_name', 'last_review', 'reviews_per_month']].head()

cleaned_missing, dtypes, sample_rows

(id                                0
 name                              0
 host_id                           0
 host_name                         0
 neighbourhood_group               0
 neighbourhood                     0
 latitude                          0
 longitude                         0
 room_type                         0
 price                             0
 minimum_nights                    0
 number_of_reviews                 0
 last_review                       0
 reviews_per_month                 0
 calculated_host_listings_count    0
 availability_365                  0
 dtype: int64,
 id                                         int64
 name                                      object
 host_id                                    int64
 host_name                                 object
 neighbourhood_group                       object
 neighbourhood                             object
 latitude                                 float64
 longitude                                f

# Duplicate Removal:
Identifying and eliminating duplicate records to maintain data
uniqueness.

In [6]:
df.duplicated().sum()

np.int64(0)

# Standardization:
Consistent formatting and units across the dataset for accurate analysis.

In [3]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns for standardization (excluding ID-like columns)
numeric_cols = ['latitude', 'longitude', 'price', 'minimum_nights',
                'number_of_reviews', 'reviews_per_month',
                'calculated_host_listings_count', 'availability_365']

# Initialize the scaler
scaler = StandardScaler()

# Apply standardization
standardized_data = scaler.fit_transform(df[numeric_cols])

# Convert the result back to DataFrame for readability
standardized_df = pd.DataFrame(standardized_data, columns=[f"{col}_scaled" for col in numeric_cols])

# Show summary statistics of the scaled features
standardized_summary = standardized_df.describe()

standardized_df.head(), standardized_summary

(   latitude_scaled  longitude_scaled  price_scaled  minimum_nights_scaled  \
 0        -1.466526         -0.454489      0.033929              -0.280022   
 1         0.463475         -0.698632      0.419827              -0.280022   
 2        -0.781851         -0.184433     -0.270728              -0.280022   
 3         1.279811          0.153298     -0.316426               0.237679   
 4         0.355272         -0.510814      0.292887              -0.164977   
 
    number_of_reviews_scaled  reviews_per_month_scaled  \
 0                 -0.421236                 -0.692221   
 1                  0.325873                 -0.591055   
 2                  4.995303                  1.944025   
 3                 -0.421236                 -0.757680   
 4                  0.927711                 -0.466087   
 
    calculated_host_listings_count_scaled  availability_365_scaled  
 0                               0.031775                 1.930781  
 1                              -0.120343 

latitude / longitude → Geographic coordinates are bounded by the city’s geography; outlier handling is more about geographic filtering (e.g., outside NYC) than IQR.

availability_365 → Already bounded between 0 and 365, and values near extremes are valid (0 = unavailable, 365 = fully available).

calculated_host_listings_count → Could be skewed, but very high values may be legitimate for property managers.

# Outlier Detection:
Identifying and addressing outliers that may skew analysis or model
performance.

In [7]:
import numpy as np

# Use IQR method to detect outliers for selected numeric features
outlier_flags = {}
outlier_counts = {}

for col in ['price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
    outlier_flags[col] = outliers
    outlier_counts[col] = outliers.sum()

outlier_counts

{'price': np.int64(2077),
 'minimum_nights': np.int64(4464),
 'number_of_reviews': np.int64(4237),
 'reviews_per_month': np.int64(1793)}